### The "genesis" version: the db is in-development and no *consolidation* tables had been created; the resulting tables from civiActivityReport serve as the consolidation and stakjob source tables and there is no need to run the status and type stored procedures because there is no update to them
This python file serves as an orchestration program to be run **after** *civiActivityReport.ipynb*.
Components:
1. rename (by way of copies) the two output tables from civiActivityReport.ipynb to consolidated_mem_type and consolidated_mem_status
    This is done in order to preserve the original import table names, which should be deleted manually later
2. call the stored procedure to create the stack_job table: *stored_procedure_create_stack_job.sql

In [4]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import itertools
import json
from container_credentials import return_credentials
import sqlalchemy

In [5]:
# DEFINE THE DATABASE CREDENTIALS

cred_dict = return_credentials()

user = cred_dict['user'] 
password = cred_dict['pass'] 
host = cred_dict['host'] 
port = cred_dict['port'] 
database = cred_dict['database']

''' legacy code hard coded the credentials
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'
'''

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		# working w/engines: https://docs.sqlalchemy.org/en/20/core/engines_connections.html
		engine = get_connection() #engine should be created just once, and can manage several DBAPI connections
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)

Connection to the 172.17.0.2 for user root created successfully.


In [14]:
#make copies of the two output tables from the .ipynb <- change table name to a generic name to be consumed by the stored procedures
#create copies of two tables at a time (in order) "new_import" suffix
def copy_rename(type_table: str, status_table: str, type_table_replacement_name = None, status_table_replacement_name = None):
    #a CORE approach
    #type_table = 'mem_type_'
    #status_table = 'mem_status_'
    #want to limit the scope of the of our use of this object to a specific context, so we use Python's context manager "with"
    with engine.connect() as conn: #interacting w/db through Connection class

        if type_table_replacement_name is None:
            conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS mem_type_new_import"))
            conn.execute(sqlalchemy.text(f"CREATE TABLE mem_type_new_import LIKE {type_table}"))
            conn.execute(sqlalchemy.text(f"INSERT INTO mem_type_new_import SELECT * FROM {type_table}"))
            conn.commit()
        elif type_table_replacement_name is not None:
            conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS {type_table_replacement_name}"))
            conn.execute(sqlalchemy.text(f"CREATE TABLE {type_table_replacement_name} LIKE {type_table}"))
            conn.execute(sqlalchemy.text(f"INSERT INTO {type_table_replacement_name} SELECT * FROM {type_table}"))
            conn.commit()
        
        if status_table_replacement_name is None:
            conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS mem_status_new_import"))
            conn.execute(sqlalchemy.text(f"CREATE TABLE mem_status_new_import LIKE {status_table}"))
            conn.execute(sqlalchemy.text(f"INSERT INTO mem_status_new_import SELECT * FROM {status_table}"))
            conn.commit()
        elif status_table_replacement_name is not None:
            conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS {status_table_replacement_name}"))
            conn.execute(sqlalchemy.text(f"CREATE TABLE {status_table_replacement_name} LIKE {status_table}"))
            conn.execute(sqlalchemy.text(f"INSERT INTO {status_table_replacement_name} SELECT * FROM {status_table}"))
            conn.commit()

        else:
            raise SyntaxError("none of the four conditions for copy_rename were encountered")

In [19]:
def qa_copy_rename(type_table: str, status_table: str, type_table_replacement_name, status_table_replacement_name):
        #verify that table lengths/shapes are the same after the import
        from sqlalchemy import text
        with engine.connect() as conn:
            type_orig = conn.execute(text(f"SELECT COUNT(*) FROM {type_table}")).scalar()
            status_orig = conn.execute(text(f"SELECT COUNT(*) FROM {status_table}")).scalar()
            type_post_import = conn.execute(text(f"SELECT COUNT(*) FROM {type_table_replacement_name}")).scalar()
            status_post_import = conn.execute(text(f"SELECT COUNT(*) FROM {status_table_replacement_name}")).scalar()
        
        if type_orig == type_post_import and status_orig == status_post_import:
            return 'tables are of expected type'
        else:
            raise RuntimeError("table sizes not as expected")

In [12]:
#create variables for the import tables (will be used at least twice in the program)
#expected that the 'type' and 'status' table names already follow the table_name_date convention
mem_type_import = 'mem_type_1007_ts'
mem_status_import = 'mem_status_1007_ts'

In [15]:
# if I want to handle errors, the relative error category is "programming-time error"
copy_rename(mem_type_import,mem_status_import,'consolidated_mem_type','consolidated_mem_status')

In [20]:
#check copy_rename
qa_copy_rename(mem_type_import,mem_status_import,'consolidated_mem_type','consolidated_mem_status')

'tables are of expected type'

The only stored procedure necessary is the **stackjob_creations** procedure. Once that is run, the db will be equipped with consolidated_mem_type, consolidated_mem_status and stack_job2

In [21]:
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("show procedure status where definer LIKE '%root%'"))
    lista = [i[1] for i in result.all()]

print(lista)    

['stackjob_creations', 'status_table_create', 'test_parameter', 'type_table_create']


In [22]:
#TODO call the stack_job stored procedure <- the final input table to the active accounts study
connection = engine.raw_connection()
try:
    cursor_obj = connection.cursor()
    cursor_obj.callproc("stackjob_creations")
    cursor_obj.close()
    connection.commit()
finally:
    connection.close()

In [42]:
#TODO add some QA of the new stack_job2 table (from the stackjob_creation stored procedurei)
